In [1]:
import pandas as pd
import re
import sentencepiece as spm

from utils.processing_utils import read_gffs
from utils.tokenizer_utils import train_sentencepiece_tokenizer, compression_ratio

In [2]:
gff_file_path = "ncbi/fungi/GCF_000146045/GCF_000146045.2_R64_genomic.gff"
genome_file_path = "ncbi/fungi/GCF_000146045/GCF_000146045.2_R64_genomic.fna"
output_file_path = "ncbi/fungi/GCF_000146045/joined_cds_sequences.fasta"

cdss = read_gffs(gff_file_path, genome_file_path, output_file_path)
cdss = [str(c.seq).upper() for c in cdss]

cds_file = "ncbi/fungi/GCF_000146045/joined_cds_sequences.txt"
with open(cds_file,'w') as f:
    f.write('\n'.join(cdss))
    

In [ ]:
train_sentencepiece_tokenizer(cds_file, 
                              'ncbi/fungi/GCF_000146045/tokenizer', 
                              10000)

In [5]:
tokenizer = spm.SentencePieceProcessor(model_file='ncbi/fungi/GCF_000146045/tokenizer.model')

toks = []
for t in tokenizer.encode(cdss):
    toks+=t
    
    
vocab = pd.read_csv('ncbi/fungi/GCF_000146045/tokenizer.vocab',sep='\t',header=None)
dfv = vocab
vocab = vocab[0].tolist()
vocab = [re.sub('[^ACCT]','',v) for v in vocab]


compression_ratio(''.join(cdss),toks,vocab)

(129070, 15900564, 17650128, 0.9081879746141218)